# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2023 12:16PM,264665,22,650009,"NBTY Global, Inc.",Released
1,Jun 13 2023 12:03PM,264664,20,9568691,Gutenberg,Released
2,Jun 13 2023 12:03PM,264660,19,9531575-BO,Eywa Pharma Inc.,Executing
3,Jun 13 2023 12:03PM,264660,19,9551489,Eywa Pharma Inc.,Released
4,Jun 13 2023 12:03PM,264660,19,9551500,Eywa Pharma Inc.,Released
5,Jun 13 2023 11:59AM,264663,15,PNC459193B,"Person & Covey, Inc.",Released
6,Jun 13 2023 11:59AM,264663,15,PNC459201A,"Person & Covey, Inc.",Released
7,Jun 13 2023 11:59AM,264663,15,PNC459202A,"Person & Covey, Inc.",Released
8,Jun 13 2023 11:59AM,264662,10,9568689,Eywa Pharma Inc.,Released
9,Jun 13 2023 11:54AM,264661,15,9567502,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,264661,Released,28
23,264662,Released,1
24,264663,Released,3
25,264664,Released,1
26,264665,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
264661,NaN,NaN,28.0
264662,NaN,NaN,1.0
264663,NaN,NaN,3.0
264664,NaN,NaN,1.0
264665,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264559,0.0,27.0,1.0
264604,0.0,1.0,0.0
264620,5.0,10.0,3.0
264622,0.0,1.0,0.0
264627,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
264559,0,27,1
264604,0,1,0
264620,5,10,3
264622,0,1,0
264627,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264559,0,27,1
1,264604,0,1,0
2,264620,5,10,3
3,264622,0,1,0
4,264627,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,264559,,27,1
1,264604,,1,
2,264620,5,10,3
3,264622,,1,
4,264627,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2023 12:16PM,264665,22,"NBTY Global, Inc."
1,Jun 13 2023 12:03PM,264664,20,Gutenberg
2,Jun 13 2023 12:03PM,264660,19,Eywa Pharma Inc.
5,Jun 13 2023 11:59AM,264663,15,"Person & Covey, Inc."
8,Jun 13 2023 11:59AM,264662,10,Eywa Pharma Inc.
9,Jun 13 2023 11:54AM,264661,15,"Brookfield Pharmaceuticals, LLC"
37,Jun 13 2023 11:52AM,264659,15,"Alliance Pharma, Inc."
41,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd"
67,Jun 13 2023 11:32AM,264657,10,"Nextsource Biotechnology, LLC"
68,Jun 13 2023 11:31AM,264656,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 13 2023 12:16PM,264665,22,"NBTY Global, Inc.",,,1
1,Jun 13 2023 12:03PM,264664,20,Gutenberg,,,1
2,Jun 13 2023 12:03PM,264660,19,Eywa Pharma Inc.,,1,2
3,Jun 13 2023 11:59AM,264663,15,"Person & Covey, Inc.",,,3
4,Jun 13 2023 11:59AM,264662,10,Eywa Pharma Inc.,,,1
5,Jun 13 2023 11:54AM,264661,15,"Brookfield Pharmaceuticals, LLC",,,28
6,Jun 13 2023 11:52AM,264659,15,"Alliance Pharma, Inc.",,,4
7,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd",,15,11
8,Jun 13 2023 11:32AM,264657,10,"Nextsource Biotechnology, LLC",,,1
9,Jun 13 2023 11:31AM,264656,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 12:16PM,264665,22,"NBTY Global, Inc.",1,,
1,Jun 13 2023 12:03PM,264664,20,Gutenberg,1,,
2,Jun 13 2023 12:03PM,264660,19,Eywa Pharma Inc.,2,1,
3,Jun 13 2023 11:59AM,264663,15,"Person & Covey, Inc.",3,,
4,Jun 13 2023 11:59AM,264662,10,Eywa Pharma Inc.,1,,
5,Jun 13 2023 11:54AM,264661,15,"Brookfield Pharmaceuticals, LLC",28,,
6,Jun 13 2023 11:52AM,264659,15,"Alliance Pharma, Inc.",4,,
7,Jun 13 2023 11:34AM,264654,19,"AdvaGen Pharma, Ltd",11,15,
8,Jun 13 2023 11:32AM,264657,10,"Nextsource Biotechnology, LLC",1,,
9,Jun 13 2023 11:31AM,264656,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 12:16PM,264665,22,"NBTY Global, Inc.",1,,
1,Jun 13 2023 12:03PM,264664,20,Gutenberg,1,,
2,Jun 13 2023 12:03PM,264660,19,Eywa Pharma Inc.,2,1,
3,Jun 13 2023 11:59AM,264663,15,"Person & Covey, Inc.",3,,
4,Jun 13 2023 11:59AM,264662,10,Eywa Pharma Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 12:16PM,264665,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 13 2023 12:03PM,264664,20,Gutenberg,1.0,NaN,NaN
2,Jun 13 2023 12:03PM,264660,19,Eywa Pharma Inc.,2.0,1.0,NaN
3,Jun 13 2023 11:59AM,264663,15,"Person & Covey, Inc.",3.0,NaN,NaN
4,Jun 13 2023 11:59AM,264662,10,Eywa Pharma Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 13 2023 12:16PM,264665,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 13 2023 12:03PM,264664,20,Gutenberg,1.0,0.0,0.0
2,Jun 13 2023 12:03PM,264660,19,Eywa Pharma Inc.,2.0,1.0,0.0
3,Jun 13 2023 11:59AM,264663,15,"Person & Covey, Inc.",3.0,0.0,0.0
4,Jun 13 2023 11:59AM,264662,10,Eywa Pharma Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2117214,31.0,10.0,0.0
15,1058542,36.0,27.0,0.0
16,793872,1.0,2.0,0.0
19,793934,16.0,26.0,5.0
20,264664,1.0,0.0,0.0
22,529287,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2117214,31.0,10.0,0.0
1,15,1058542,36.0,27.0,0.0
2,16,793872,1.0,2.0,0.0
3,19,793934,16.0,26.0,5.0
4,20,264664,1.0,0.0,0.0
5,22,529287,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,10.0,0.0
1,15,36.0,27.0,0.0
2,16,1.0,2.0,0.0
3,19,16.0,26.0,5.0
4,20,1.0,0.0,0.0
5,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,15,Released,36.0
2,16,Released,1.0
3,19,Released,16.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,22
Status,,,,,,
Completed,0.0,0.0,0.0,5.0,0.0,0.0
Executing,10.0,27.0,2.0,26.0,0.0,1.0
Released,31.0,36.0,1.0,16.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,22
0,Completed,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,10.0,27.0,2.0,26.0,0.0,1.0
2,Released,31.0,36.0,1.0,16.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,22
0,Completed,0.0,0.0,0.0,5.0,0.0,0.0
1,Executing,10.0,27.0,2.0,26.0,0.0,1.0
2,Released,31.0,36.0,1.0,16.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()